In [3]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = """あなたはAI言語モデルのアシスタントです。
あなたのタスクは、与えられたユーザーの質問に対して、ベクトルデータベースから関連するドキュメントを取得するために、
5つの異なるバージョンを生成することです。
ユーザーの質問に対する複数の視点を提供することで、距離ベースの類似検索の制限を克服することを目的としています。
これらの代替質問を改行で区切って提供してください。代替質問のみを出力してください。

元の質問: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

In [9]:
generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [10]:
generate_queries.invoke({"question": "RAGとはなんですか？"})

['RAGの意味は何ですか？',
 'RAGはどのように使用されますか？',
 'RAGの主な特徴は何ですか？',
 'RAGはどのように進化してきましたか？',
 'RAGの応用例はありますか？']

In [13]:
generate_queries.invoke({"question": "RAGのテクニックにはどのようなものがありますか？"})

['RAGテクニックの種類は何ですか？',
 'RAG手法にはどんな種類がありますか？',
 'RAGの手法にはどんなものがありますか？',
 'RAGの手法にはどんな種類がありますか？',
 'RAGのテクニックには何が含まれていますか？']

In [18]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


question = "RAGとはなんですか？"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
print(len(docs))
docs

9


[Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの品質計測\n\n「改善は計測から」なので、どのように評価するかは改善前に決めておく必要があります。\n補足として、こちらの評価指標については、RAG評価ツールの「RAGAS」から、もう少し多くの指標が提案されています。[4]'),
 Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='RAGの仕組み\nこの記事をご覧のエンジニアの方であれば、既にご存知の内容かと思います。以下、チートシートの引用です。\n「RAGでは、ユーザーが質問すると、まず外部データベースから関連するドキュメントを取得する。そのドキュメントと元々のユーザーの質問がセットされ、LLMに渡される。LLMは、この内容をもとに回答を生成する」\n非常にシンプルな内容なので、この図と一緒に説明すれば、ビジネスサイドの方でも理解してもらえます。私個人的にも、似たような図を使って顧客や社内ビジネスサイドに説明していて、必ず理解してもらえる印象です。\n\nRAGの良さはこのシンプルさ、始めやすさなのですが、始めのうちは、なかなか思い通りの回答が得られません。この回答精度を上げようとすると、かなりの苦難が待っています...'),
 Document(metadata={'source': 'https://zenn.dev/knowledgesense/articles/47de9ead8029ba', 'title': 'Zenn'}, page_content='/ 好きな言葉は「実験と学習」/ 最新の生成AI 事情に少し詳しいです目次はじめにこの記事は何対象読者本題RAGテクニック集（チートシート）RAGとは。なぜ必要なのか？RAGの仕組みRAGの精度向上のために必要なことRAGを使ってできることRAGの品質計測まとめZennエンジニアのための情報共有コミュニティAboutZ

In [20]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """以下のコンテキストに基づいて以下の質問に答えてください。

{context}

質問: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0)
final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"question": question}))

回答: RAGは、ファイルを参照して回答できるLLM（例えばChatGPT）を作成するための方法であり、通常のLLMでは回答の正確性を向上させるのに限界があるため、RAGが必要とされています。
